<a href="https://colab.research.google.com/github/Nachi2006/MLREPO/blob/main/Deepfake_detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.applications import InceptionResNetV2, Xception, EfficientNetB7
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model, load_model
import requests
from IPython.display import display, Image
import ipywidgets as widgets
from PIL import Image as PILImage
import io
import gc
import os
import json
from datetime import datetime
import shutil

class LearningDeepfakeDetector:
    def __init__(self):
        self.input_size = (299, 299)
        self.dataset_dir = 'deepfake_dataset'
        self.metadata_file = os.path.join(self.dataset_dir, 'metadata.json')
        self.initialize_dataset_directory()
        self.models = self._build_ensemble()

    def initialize_dataset_directory(self):
        """Initialize dataset directory structure"""
        # Create main directory
        os.makedirs(self.dataset_dir, exist_ok=True)

        # Create subdirectories for real and fake images
        os.makedirs(os.path.join(self.dataset_dir, 'real'), exist_ok=True)
        os.makedirs(os.path.join(self.dataset_dir, 'fake'), exist_ok=True)

        # Initialize or load metadata
        if not os.path.exists(self.metadata_file):
            self.metadata = {
                'images': {},
                'statistics': {
                    'total_images': 0,
                    'real_images': 0,
                    'fake_images': 0
                }
            }
            self._save_metadata()
        else:
            self._load_metadata()

    def _load_metadata(self):
        """Load metadata from file"""
        try:
            with open(self.metadata_file, 'r') as f:
                self.metadata = json.load(f)
        except:
            print("Error loading metadata, initializing new metadata")
            self.metadata = {
                'images': {},
                'statistics': {
                    'total_images': 0,
                    'real_images': 0,
                    'fake_images': 0
                }
            }

    def _save_metadata(self):
        """Save metadata to file"""
        with open(self.metadata_file, 'w') as f:
            json.dump(self.metadata, f, indent=4)

    def _build_base_model(self, model_class, name):
        """Build a base model with custom top layers"""
        base = model_class(
            weights='imagenet',
            include_top=False,
            input_shape=(*self.input_size, 3)
        )

        base.trainable = False

        inputs = Input(shape=(*self.input_size, 3))
        x = base(inputs)
        x = GlobalAveragePooling2D()(x)
        x = Dense(1024, activation='relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(512, activation='relu')(x)
        x = Dropout(0.3)(x)
        outputs = Dense(1, activation='sigmoid', name=f'{name}_output')(x)

        model = Model(inputs=inputs, outputs=outputs, name=name)
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss='binary_crossentropy',
            metrics=['accuracy']
        )
        return model

    def _build_ensemble(self):
        """Build an ensemble of models"""
        print("Building ensemble of models...")
        models = {
            'inception': self._build_base_model(InceptionResNetV2, 'inception'),
            'xception': self._build_base_model(Xception, 'xception'),
            'efficientnet': self._build_base_model(EfficientNetB7, 'efficientnet')
        }
        print("Ensemble built successfully!")
        return models

    def save_to_dataset(self, image, is_deepfake, confidence, filename=None):
        """Save image to dataset with metadata"""
        try:
            # Generate filename if not provided
            if filename is None:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"image_{timestamp}.jpg"

            # Determine subdirectory
            subdir = 'fake' if is_deepfake else 'real'
            save_path = os.path.join(self.dataset_dir, subdir, filename)

            # Save image
            if isinstance(image, np.ndarray):
                cv2.imwrite(save_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
            elif isinstance(image, str) and image.startswith('http'):
                response = requests.get(image)
                with open(save_path, 'wb') as f:
                    f.write(response.content)

            # Update metadata
            self.metadata['images'][filename] = {
                'path': save_path,
                'is_deepfake': is_deepfake,
                'confidence': confidence,
                'date_added': datetime.now().isoformat(),
                'verified': False
            }

            # Update statistics
            self.metadata['statistics']['total_images'] += 1
            if is_deepfake:
                self.metadata['statistics']['fake_images'] += 1
            else:
                self.metadata['statistics']['real_images'] += 1

            # Save metadata
            self._save_metadata()

            return True
        except Exception as e:
            print(f"Error saving to dataset: {e}")
            return False

    def preprocess_image(self, image):
        """Preprocess image for prediction"""
        try:
            if isinstance(image, str):
                if image.startswith('http'):
                    response = requests.get(image)
                    img_array = np.asarray(bytearray(response.content), dtype=np.uint8)
                    img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)
                else:
                    img = cv2.imread(image)
            elif isinstance(image, np.ndarray):
                img = image.copy()
            else:
                raise ValueError("Unsupported image format")

            if img is None:
                raise ValueError("Could not read image")

            # Convert to RGB
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Store original for dataset
            original_img = img.copy()

            # Resize for model
            img = cv2.resize(img, self.input_size)

            # Create preprocessed versions for each model
            preprocessed = {
                'inception': tf.keras.applications.inception_resnet_v2.preprocess_input(img.copy()),
                'xception': tf.keras.applications.xception.preprocess_input(img.copy()),
                'efficientnet': tf.keras.applications.efficientnet.preprocess_input(img.copy())
            }

            return {name: np.expand_dims(img, 0) for name, img in preprocessed.items()}, original_img

        except Exception as e:
            print(f"Error preprocessing image: {e}")
            return None, None

    def detect_deepfake(self, image, save_to_dataset=True):
        """Detect deepfake and optionally save to dataset"""
        try:
            # Preprocess image
            preprocessed_images, original_img = self.preprocess_image(image)
            if preprocessed_images is None:
                return {"error": "Failed to process image"}

            # Get predictions from each model
            predictions = {}
            for name, model in self.models.items():
                pred = model.predict(preprocessed_images[name], verbose=0)[0][0]
                predictions[name] = float(pred)

            # Calculate weighted ensemble prediction
            weights = {
                'inception': 0.4,
                'xception': 0.3,
                'efficientnet': 0.3
            }

            weighted_pred = sum(pred * weights[name] for name, pred in predictions.items())

            # Calculate confidence metrics
            prediction_std = np.std(list(predictions.values()))
            model_agreement = 1 - prediction_std

            # Final confidence calculation
            confidence = weighted_pred
            confidence = np.clip(confidence, 0, 1)

            # Prepare result
            result = {
                "is_deepfake": bool(confidence > 0.5),
                "confidence": confidence * 100,
                "model_agreement": model_agreement * 100,
                "model_predictions": {
                    name: float(pred * 100) for name, pred in predictions.items()
                }
            }

            # Save to dataset if requested
            if save_to_dataset and original_img is not None:
                saved = self.save_to_dataset(
                    original_img,
                    result["is_deepfake"],
                    result["confidence"]
                )
                result["saved_to_dataset"] = saved

            return result
        except Exception as e:
            return {"error": f"Detection failed: {str(e)}"}

    def get_dataset_stats(self):
        """Get dataset statistics"""
        return {
            "Total Images": self.metadata['statistics']['total_images'],
            "Real Images": self.metadata['statistics']['real_images'],
            "Fake Images": self.metadata['statistics']['fake_images']
        }

class SimpleUI:
    def __init__(self):
        print("Initializing Learning Deepfake Detector...")
        self.detector = LearningDeepfakeDetector()
        self.setup_ui()
        print("Ready!")
        self._display_stats()

    def _display_stats(self):
        """Display current dataset statistics"""
        stats = self.detector.get_dataset_stats()
        print("\nDataset Statistics:")
        for key, value in stats.items():
            print(f"{key}: {value}")

    def setup_ui(self):
        self.file_upload = widgets.FileUpload(
            accept='image/*',
            multiple=False,
            description='Upload Image'
        )

        self.url_input = widgets.Text(
            description='Image URL:',
            placeholder='Enter URL and press Enter'
        )

        self.save_checkbox = widgets.Checkbox(
            value=True,
            description='Save to dataset',
            disabled=False
        )

        self.clear_button = widgets.Button(
            description='Clear'
        )

        self.output = widgets.Output()

        self.file_upload.observe(self._handle_upload, names='value')
        self.url_input.on_submit(self._handle_url)
        self.clear_button.on_click(self._handle_clear)

        display(widgets.VBox([
            self.file_upload,
            self.url_input,
            self.save_checkbox,
            self.clear_button,
            self.output
        ]))

    def _handle_upload(self, change):
        if not change.new:
            return

        self.output.clear_output()

        try:
            file_info = next(iter(change.new.values()))
            image_data = file_info['content']

            image_array = np.frombuffer(image_data, np.uint8)
            img = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

            if img is None:
                raise ValueError("Could not decode image")

            with self.output:
                pil_img = PILImage.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                display(pil_img)

                result = self.detector.detect_deepfake(img, save_to_dataset=self.save_checkbox.value)
                self._display_result(result)
                self._display_stats()

        except Exception as e:
            with self.output:
                print(f"Error: {str(e)}")
        finally:
            self.file_upload.value.clear()
            gc.collect()

    def _handle_url(self, widget):
        url = widget.value.strip()
        if not url:
            return

        self.output.clear_output()

        try:
            with self.output:
                display(Image(url=url, width=400))
                result = self.detector.detect_deepfake(url, save_to_dataset=self.save_checkbox.value)
                self._display_result(result)
                self._display_stats()

        except Exception as e:
            with self.output:
                print(f"Error: {str(e)}")
        finally:
            widget.value = ""
            gc.collect()

    def _display_result(self, result):
        """Display detection results"""
        if "error" in result:
            print(f"Error: {result['error']}")
            return

        print(f"\nResult: {'DEEPFAKE' if result['is_deepfake'] else 'AUTHENTIC'}")
        print(f"Confidence: {result['confidence']:.2f}%")
        print(f"Model Agreement: {result['model_agreement']:.2f}%")

        print("\nIndividual Model Predictions:")
        for model, pred in result['model_predictions'].items():
            print(f"- {model.title()}: {pred:.2f}%")

        if result.get("saved_to_dataset"):
            print("\n✅ Image saved to dataset")

    def _handle_clear(self, b):
        self.output.clear_output()
        self.url_input.value = ""
        if hasattr(self.file_upload, 'value'):
            self.file_upload.value.clear()
        gc.collect()
        self._display_stats()

# Create and display the UI
print("Installing required packages...")
print("This may take a few minutes...")
ui = SimpleUI()




Installing required packages...
This may take a few minutes...
Initializing Learning Deepfake Detector...
Building ensemble of models...
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Ensemble built successfully!


Ready!

Dataset Statistics:
Total Images: 0
Real Images: 0
Fake Images: 0
